# Example of using the Borehole3D class

## Imports

In [28]:
from striplog import Component, Lexicon #, Striplog, Decor, Legend 
from core.omf import Borehole3D, striplog_legend_to_omf_legend 
from utils.io import boreholes_from_files,gdf_viewer, df_from_sources, gdf_merger
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base 
from utils.lexicon.Lexicon_FR_updated import LEXICON
from os import remove
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR
import pandas as pd
import matplotlib.pyplot as plt

## Reading of CSV files in a dir

In [29]:
search_dir=ROOT_DIR+'/CF_data/synthese/Result_traitem/'
#cols=['ID','X','Y','Z','Long_Pz','Diam_Pz_int']
cols=None
filename='source_Boreholes'

In [30]:
df_all=df_from_sources(search_dir, filename, cols, verbose=False)

--> base_Memoris3/source_Boreholes.csv: (148 lines)
--> profils_sols_donnees_forages/source_Boreholes.csv: (42 lines)

The overall dataframe contains 190 lines. 2 files used


In [31]:
cols=None
filename='source_Lithology'

df_litho=df_from_sources(search_dir, filename, cols, verbose=False)

--> base_Memoris3/source_Lithology.csv: (1629 lines)
--> profils_sols_donnees_forages/source_Lithology.csv: (54 lines)

The overall dataframe contains 1683 lines. 2 files used


In [32]:
merge_df, error_df = gdf_merger(df_litho, df_all, col='ID', how='inner')

In [33]:
merge_df = merge_df[['ID','X','Y','Z','Long_Pz','Diam_Pz_int','Description','Litho_top','Litho_base']]
gdf_viewer(merge_df, rows=5)

Rows : 72, columns : 9


interactive(children=(IntSlider(value=5, description='rows', max=72, min=5, readout=False), IntSlider(value=9,…

## Data Checking

In [34]:
dist_crit=1
df=merge_df

indexes = []
for i in range(0,len(df)-1):
    if 'geometry' in df.columns:
        d=df.loc[i, 'geometry'].distance(df.loc[i+1, 'geometry'])
        if d<dist_crit or d>1e+4:
            indexes.append(i)
            #print(f"{df.loc[i,'ID']} <-> {df.loc[i+1,'ID']} : {d:.2} meters")

if 'geometry' in df.columns:
    print('\n',f'{len(indexes)}/{len(df)} elements with probably bad XY')
    print(f'\nIndexes: {indexes}\n\nNAN_indexes: {df.query("X!=X").index}')
else:
    print('No geometry column in the dataframe')

No geometry column in the dataframe


## Lithology extraction and test

In [35]:
lexicon = Lexicon(LEXICON)

In [36]:
desc_col = 'Description'
df = merge_df

In [37]:
comp_list = []
for d in df[desc_col]:
    comp_list.append(Component.from_text(d, lexicon))
df['Component']=comp_list

In [38]:
for i in range(len(df)):
    d=df.loc[i, desc_col]
    Component.from_text(d, lexicon)

In [39]:
idx=65

print(f"{type(df['Component'][idx])}\n\n idx {idx}: [{df['ID'][idx]}] --> "
      f"{df['Description'][idx]}")
df['Component'][idx] # TODO : need some modifications on colour, grainsize lexicons

<class 'striplog.component.Component'>

 idx 65: [F15bM] --> Vers 4 m, limon argileux verdâtre puis brun. Entre 4,6 m et 4,8 m strate graveleuse avec irrisation et pollution nette de l'eau souterraine.


lithology,limon
modifier,argileux
colour,verdâtre


In [40]:
# problem with this description above : there are two intervals in one description --> 
# data incoherence (unfortunately)

## Create boreholes and components objects from dataframe

In [41]:
boreholes, components, link_intv_comp= boreholes_from_files([df], verbose=False, diam_field='Diam_Pz_int', 
                                            length_field='Long_Pz', litho_field='Description',
                                           litho_top='Litho_top', litho_base='Litho_base',
                                           lexicon=lexicon)#, x=list(df_all.X), y=list(df_all.Y), verbose=False)


Dataframe 0 processing...
{'F7': Striplog(9 Intervals, start=0.0, stop=8.2)}
{'F8': Striplog(9 Intervals, start=0.0, stop=7.0)}
{'F1aM': Striplog(3 Intervals, start=0.0, stop=2.4)}
{'F2M': Striplog(3 Intervals, start=0.0, stop=6.0)}
{'F3M': Striplog(4 Intervals, start=0.0, stop=3.3)}
{'F4M': Striplog(5 Intervals, start=0.0, stop=6.0)}
{'F5M': Striplog(4 Intervals, start=0.0, stop=6.0)}
{'F6M': Striplog(4 Intervals, start=0.0, stop=4.8)}
{'F7aM': Striplog(1 Intervals, start=0.0, stop=0.3)}
{'F7bM': Striplog(1 Intervals, start=0.0, stop=0.3)}
{'F8M': Striplog(1 Intervals, start=0.0, stop=0.3)}
{'F9aM': Striplog(1 Intervals, start=0.0, stop=0.4)}
{'F9bM': Striplog(1 Intervals, start=0.0, stop=0.6)}
{'F10M': Striplog(2 Intervals, start=0.0, stop=2.0)}
{'F11M': Striplog(2 Intervals, start=0.0, stop=3.0)}
{'F12M': Striplog(5 Intervals, start=0.0, stop=4.8)}
{'F13M': Striplog(3 Intervals, start=0.0, stop=4.8)}
{'F14M': Striplog(3 Intervals, start=0.0, stop=2.8)}
{'F15aM': Striplog(1 Interval

In [42]:
len(boreholes), len(components)

(26, 55)

### Saving in a database

In [43]:
remove('tmp_files/tfe_orm_db.db')

In [44]:
engine = create_engine('sqlite:///tmp_files/tfe_orm_db.db', echo=True)

In [45]:
Base.metadata.create_all(engine)

2021-04-04 00:34:47,195 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-04-04 00:34:47,196 INFO sqlalchemy.engine.base.Engine ()
2021-04-04 00:34:47,197 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-04-04 00:34:47,198 INFO sqlalchemy.engine.base.Engine ()
2021-04-04 00:34:47,199 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2021-04-04 00:34:47,199 INFO sqlalchemy.engine.base.Engine ()
2021-04-04 00:34:47,200 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boreholes")
2021-04-04 00:34:47,201 INFO sqlalchemy.engine.base.Engine ()
2021-04-04 00:34:47,202 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Positions")
2021-04-04 00:34:47,202 INFO sqlalchemy.engine.base.Engine ()
2021-04-04 00:34:47,203 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Positions")
2021-04-04 00:34:47,204 INFO sqlalchemy.engine.base.Engine ()
2021-04-04

## Add differents objects into the project

In [46]:
Session = sessionmaker(bind=engine)
session = Session()

In [47]:
p = Project(session)
p.add_components(components)

2021-04-04 00:34:47,245 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-04-04 00:34:47,246 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-04-04 00:34:47,247 INFO sqlalchemy.engine.base.Engine ()
2021-04-04 00:34:47,257 INFO sqlalchemy.engine.base.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2021-04-04 00:34:47,258 INFO sqlalchemy.engine.base.Engine ((0, 'Remblai, scories'), (1, 'Limon, bariolé, vert, hap'), (2, 'Remblai, scories, verdâtre, huile'), (3, 'Remblai, scories, hap'), (4, 'Remblai, scories, verdâtre, hap'), (5, 'Limon, scories, verdâtre'), (6, 'Limon, gris, traces, hap'), (7, 'Sable, jaune')  ... displaying 10 of 55 total bound parameter sets ...  (53, 'Remblais, gris foncé, centimétrique'), (54, 'Remblais, hétérogènes, sableuse, sableuse, centimétrique'))
2021-04-04 00:34:47,260 INFO sqlalchemy.engine.base.Engi

In [48]:
for bh in boreholes:
    p.add_borehole(bh)

2021-04-04 00:34:47,269 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id, length, diameter) VALUES (?, ?, ?)
2021-04-04 00:34:47,270 INFO sqlalchemy.engine.base.Engine ('F7', 3.366, 0.041)
2021-04-04 00:34:47,272 INFO sqlalchemy.engine.base.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-04-04 00:34:47,273 INFO sqlalchemy.engine.base.Engine ((0, 0.0, 0.0, 0.0, 153049.921, 122636.217), (1, 0.4, 0.4, 0.4, 153049.921, 122636.217), (2, 0.4, 0.4, 0.4, 153049.921, 122636.217), (3, 1.0, 1.0, 1.0, 153049.921, 122636.217), (4, 1.0, 1.0, 1.0, 153049.921, 122636.217), (5, 2.2, 2.2, 2.2, 153049.921, 122636.217), (6, 2.2, 2.2, 2.2, 153049.921, 122636.217), (7, 3.4, 3.4, 3.4, 153049.921, 122636.217)  ... displaying 10 of 18 total bound parameter sets ...  (16, 7.7, 7.7, 7.7, 153049.921, 122636.217), (17, 8.2, 8.2, 8.2, 153049.921, 122636.217))
2021-04-04 00:34:47,275 INFO sqlalchemy.engine.base.Engine INSERT INTO "Intervals" (id, borehole,

2021-04-04 00:34:47,305 INFO sqlalchemy.engine.base.Engine (13,)
2021-04-04 00:34:47,306 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-04 00:34:47,307 INFO sqlalchemy.engine.base.Engine (14,)
2021-04-04 00:34:47,308 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-04 00:34:47,309 INFO sqlalchemy.engine.base.Engine (15,)
2021-04-04 00:34:47,310 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2021-04-04 00:34:47,376 INFO sqlalchemy.engine.base.Engine (26,)
2021-04-04 00:34:47,377 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-04 00:34:47,377 INFO sqlalchemy.engine.base.Engine (27,)
2021-04-04 00:34:47,379 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-04 00:34:47,380 INFO sqlalchemy.engine.base.Engine (28,)
2021-04-04 00:34:47,381 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2021-04-04 00:34:47,455 INFO sqlalchemy.engine.base.Engine (41,)
No given legend or incorrect format ! A default legend will be used
Borehole geometry created successfully !
2021-04-04 00:34:47,487 INFO sqlalchemy.engine.base.Engine UPDATE "Boreholes" SET length=? WHERE "Boreholes".id = ?
2021-04-04 00:34:47,488 INFO sqlalchemy.engine.base.Engine (2.4, 'F1aM')
2021-04-04 00:34:47,489 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id, length, diameter) VALUES (?, ?, ?)
2021-04-04 00:34:47,490 INFO sqlalchemy.engine.base.Engine ('F2M', 3.366, 0.041)
2021-04-04 00:34:47,491 INFO sqlalchemy.engine.base.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-04-04 00:34:47,492 INFO sqlalchemy.engine.base.Engine ((42, 0.0, 0.0, 0.0, 153049.921, 122636.217), (43, 2.4, 2.4, 2.4, 153049.921, 122636.217), (44, 2.4, 2.4, 2.4, 153049.921, 122636.217), (45, 4.0, 4.0, 4.0, 153049.921, 122636.217), (46, 4.0, 4.0, 4.0, 153049.921, 122636.217), (47, 6

2021-04-04 00:34:47,579 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-04 00:34:47,580 INFO sqlalchemy.engine.base.Engine (50,)
2021-04-04 00:34:47,582 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-04 00:34:47,583 INFO sqlalchemy.engine.base.Engine (51,)
2021-04-04 00:34:47,584 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Position

...

2021-04-04 00:34:48,307 INFO sqlalchemy.engine.base.Engine (4.8, 'F12M')
2021-04-04 00:34:48,308 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id, length, diameter) VALUES (?, ?, ?)
2021-04-04 00:34:48,309 INFO sqlalchemy.engine.base.Engine ('F13M', 4.329000000000008, 0.041)
2021-04-04 00:34:48,310 INFO sqlalchemy.engine.base.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id) VALUES (?, ?, ?, ?, ?, ?)
2021-04-04 00:34:48,310 INFO sqlalchemy.engine.base.Engine ((55, 'F13M', 0, 'Remblais pierreux de taille centimétrique à décimétrique ', 110, 111), (56, 'F13M', 1, 'Remblais divers hétérogène avec charge orange de déchets de construction de texture graveleuse. A la base des remblais, strate noire 10-20 cm de type cendrées ', 112, 113), (57, 'F13M', 2, "Alluvions limoneux argileux noir puis devient verdâtre. Pas d'odeur.", 114, 115))
2021-04-04 00:34:48,312 INFO sqlalchemy.engine.base.Engine COMMIT
2021-04-04 00:34:48,317 INFO sqlalch

2021-04-04 00:34:48,401 INFO sqlalchemy.engine.base.Engine (120,)
2021-04-04 00:34:48,402 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-04 00:34:48,402 INFO sqlalchemy.engine.base.Engine (121,)
No given legend or incorrect format ! A default legend will be used
Borehole geometry created successfully !
2021-04-04 00:34:48,444 INFO sqlalchemy.engine.base.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-04-04 00:34:48,445 INFO sqlalchemy.engine.base.Engine ((122, 0.0, 0.0, 0.0, 152883.878158, 122606.5786), (123, 0.8, 0.8, 0.8, 152883.878158, 122606.5786))
2021-04-04 00:34:48,446 INFO sqlalchemy.engine.base.Engine UPDATE "Boreholes" SET length=? WHERE "Boreholes".id = ?
2021-04-04 0

2021-04-04 00:34:48,558 INFO sqlalchemy.engine.base.Engine ('F15bM',)
2021-04-04 00:34:48,559 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-04 00:34:48,560 INFO sqlalchemy.engine.base.Engine (126,)
2021-04-04 00:34:48,561 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-04 00:34:48,562 INFO sqlalchemy.engine.base.Engine (127,)
2021-04-04 00:34:48,563 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", 

2021-04-04 00:34:48,671 INFO sqlalchemy.engine.base.Engine ('F17aM',)
2021-04-04 00:34:48,671 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-04 00:34:48,672 INFO sqlalchemy.engine.base.Engine (136,)
2021-04-04 00:34:48,673 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-04 00:34:48,673 INFO sqlalchemy.engine.base.Engine (137,)
No given legend or incorrect format ! A default legend will be used
Borehole geometry created successfully !
2021-04-04 00:34:48,704 INFO sq

2021-04-04 00:34:48,822 INFO sqlalchemy.engine.base.Engine ('F17dM',)
2021-04-04 00:34:48,823 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-04 00:34:48,823 INFO sqlalchemy.engine.base.Engine (142,)
2021-04-04 00:34:48,824 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-04 00:34:48,824 INFO sqlalchemy.engine.base.Engine (143,)
No given legend or incorrect format ! A default legend will be used
Borehole geometry created successfully !


In [49]:
p.add_link_between_components_and_intervals(link_intv_comp)

2021-04-04 00:34:48,862 INFO sqlalchemy.engine.base.Engine UPDATE "Boreholes" SET length=? WHERE "Boreholes".id = ?
2021-04-04 00:34:48,863 INFO sqlalchemy.engine.base.Engine (3.6, 'F17dM')
2021-04-04 00:34:48,866 INFO sqlalchemy.engine.base.Engine INSERT INTO "Linkintervalcomponent" (int_id, comp_id, extra_data) VALUES (?, ?, ?)
2021-04-04 00:34:48,866 INFO sqlalchemy.engine.base.Engine ((0, 7, ''), (1, 0, ''), (2, 2, ''), (3, 3, ''), (4, 4, ''), (5, 0, ''), (6, 5, ''), (7, 1, '')  ... displaying 10 of 71 total bound parameter sets ...  (70, 54, ''), (71, 54, ''))
2021-04-04 00:34:48,868 INFO sqlalchemy.engine.base.Engine COMMIT
2021-04-04 00:34:48,871 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-04-04 00:34:48,872 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-04-04 00:34:48,873 INFO sqlalchemy.engine.base.Engine ()


In [50]:
p.commit()
print('Boreholes in the project : ', len(p.boreholes))

2021-04-04 00:34:48,877 INFO sqlalchemy.engine.base.Engine COMMIT
Boreholes in the project :  26


In [51]:
p.refresh(verbose=True)

2021-04-04 00:34:48,884 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-04-04 00:34:48,885 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-04-04 00:34:48,886 INFO sqlalchemy.engine.base.Engine ()


In [52]:
session.close()

2021-04-04 00:34:48,891 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [53]:
p.boreholes_3d[0].intervals

[Interval({'top': Position({'middle': 0.0, 'upper': 0.0, 'lower': 0.0, 'x': 153049.921, 'y': 122636.217, 'units': 'm'}), 'base': Position({'middle': 0.4, 'x': 153049.921, 'y': 122636.217, 'units': 'm'}), 'description': 'Mélange de sable jaune et de cendrées goudronneuses; odeur moyenne de goudron', 'data': {}, 'components': [Component({'lithology': 'sable', 'colour': 'jaune'})]}),
 Interval({'top': Position({'middle': 0.4, 'x': 153049.921, 'y': 122636.217, 'units': 'm'}), 'base': Position({'middle': 1.0, 'x': 153049.921, 'y': 122636.217, 'units': 'm'}), 'description': 'Remblai de cendrées, scories et laitiers; forte odeur de goudron', 'data': {}, 'components': [Component({'lithology': 'remblai', 'material': 'scories'})]}),
 Interval({'top': Position({'middle': 1.0, 'x': 153049.921, 'y': 122636.217, 'units': 'm'}), 'base': Position({'middle': 2.2, 'x': 153049.921, 'y': 122636.217, 'units': 'm'}), 'description': "Remblai de cendrées et scories; présence d'un produit gras verdâtre de type

## The Borehole 3D plot using X3D for interaction within the notebook

In [54]:
p.plot3d(x3d=True)

##  ================== Draft =================

#### Codes qui peuvent aider si besoin (Brouillon)